1) define list of wav files, load model

2) for each wav file

    load first segment,
    
        get spectrogram
    
        preprocess, normalize scale...

        ([option] save data! or jump previous step because it has already been saved?)
        
        aply model
        
        get predictions
        
        get first segment annotations
    
    load 2nd segment
    
        get spectrogram
    
        preprocess, normalize scale...

        ([option] save data! or jump previous step because it has already been saved?)
    
        apply model
        
        get predictions
        
        concatenate new annotation to previous anotation file
     
     etc.. till end of wav file
         
     save single anotation file for whole wav.
    
    
3) for each prediction annotation data that becomes available, 

    run code to compute regions files for sonic visualizer!
        
        transform list of prediction matrix per segment [audiox_startSec_to_finishSec, pred_matrix]
        
    this can be done in separate/ parallel to the first step... 
    
    moved to another notebook
    
    

In [1]:
import numpy as np
import os
import librosa
from tqdm.notebook import tqdm
import csv
import tensorflow as tf
import sys
import pickle
from keras.layers import Input
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Reshape, Permute, multiply
from keras.layers import TimeDistributed, Dense, Dropout
from keras.layers import GRU, Bidirectional
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras import optimizers
from tensorflow.python.client import device_lib
from sklearn.metrics import roc_curve, auc
from keras import backend as K
import datetime

from keras.models import load_model


os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
tf.Session(config=config)

/homes/in304/manx_shearwaters/venv/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/in304/manx_shearwaters/venv/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/in304/manx_shearwaters/venv/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/in304/manx_shearwaters/venv/lib64

In [6]:
def reformat(x): # modified for single spectrogram. n =1, no labels!
    """Reformat data into a suitable format.
    
    # Arguments
        dataset: dataset in format (id, spectro, label)
        
    # Returns
        x: spectros normalised across each mel band in format (n, timesteps, mel bands, 1)
        y: labels in format (n, timesteps, 8)
    """
#     x = dataset[:, 1] 
#     x = np.stack(x) # reshape to (n, mel bands, timesteps)
    x = np.expand_dims(np.moveaxis(x, 0, 1), axis=2) # reformat x to (n, timesteps, mel bands, 1)  
#     x = np.expand_dims(np.moveaxis(x, 1, -1), axis=2)
#     y = dataset[:, 2] 
    print(x.shape)
#     y = np.moveaxis(np.stack(y), 1, -1) # reformat y to (n, timesteps, 8)

    x=np.expand_dims(x, axis=0)
    print(x.shape)
    return x


In [3]:
segment_size = 6
slide = 6
mels = 64
num_mels= 45
sample_rate = 22_050
timesteps = 259 # number of frames in a segment! =((SR* segment_size)/ hop_length)
timesteps_per_second = timesteps / segment_size

############# settings: ##################################################################
#where to save the prediction matrix to
# save_path = "/import/c4dm-datasets/animal_identification/manx_shearwaters/testing_model/"
save_path = "/homes/in304/manx_shearwaters/testing_aplying_model/

# list of wav files to make annotations
wavs_list = ['135_2A19_20190522_0050.WAV', '135_2A19_20190522_0240.WAV','135_2A19_20190522_0330.WAV'] 
path_wavs = "/import/c4dm-datasets/animal_identification/manx_shearwaters/2019_recordings/135_2A19/"
#  ALTERNATIVELY get list of wavs like this:
# wavs_list = os.listdir("/import/c4dm-datasets/animal_identification/manx_shearwaters/2019_recordings/135_2A19/")


# which model to use (path)
model_path =  "/homes/in304/manx_shearwaters/shearwaters/experiments/exp_adults_vs_chick_with_silences/2020-04-19_22:22:01.794763_train_baseline_PCEN/savedmodel.h5"



prediction_theshold = 0.5

In [4]:

# load model:

model = load_model(model_path)
#







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:

for wfile in tqdm(wavs_list, desc='load_audio'):
    prediction_annotations = []
    
    print('load audio')
    #Load audio,
    y, sr = librosa.load(path_wavs +wfile, sr=sample_rate)  #this takes a loooong time, consider doing it by segments, i.e loading each segment instead of everything and then cutting
    length = int(len(y) / sr) #seconds
    remainder = length % segment_size
    #     import pdb; pdb.set_trace()

    audio_id = wfile[:-4]
    
#   segment, cycle for each segment:
    for t in tqdm(range(0, length - remainder - segment_size, slide),
                  desc='create_spectros'):
        start = t
        stop = t + segment_size
        print('start in frames', str(sr*start))
        print('stop in frames: ', str((sr*stop)))
        current_y = y[sr*start:(sr*stop)]
        
        # create spectrogram
        spectro = librosa.feature.melspectrogram(y=current_y, sr=sr, n_mels=mels,
                                                 fmax=sr/2)
        dt_id = audio_id + '_' + str(start) + 's_to' + str(stop) + 's'
        print(dt_id)
        
#       treat segment spectrogram for network
#          PCEN:
        spectro_PCEN = librosa.pcen(spectro * (2**31))
#         further normalize, scale?
        x = reformat(spectro_PCEN)
#         subselect mel bands! didnt realize before we were doing this...new_mels = 45
        x = x[:, :, :num_mels]
        
        print(x.shape)
        
#          predict with loaded model
        predicted = model.predict(x) 
#          get binary predictions accordingly to threshold
        predicted[predicted > prediction_theshold] = 1
        predicted[predicted <= prediction_theshold] = 0
        
        prediction_annotations.append([dt_id, predicted])
        
    #save data processed for this file
    d_array = np.asarray(prediction_annotations)
    np.save(save_path+audio_id, d_array)
  

load audio


start in frames 0
stop in frames:  132300
135_2A19_20190522_0050_0s_to6s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 132300
stop in frames:  264600
135_2A19_20190522_0050_6s_to12s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 264600
stop in frames:  396900
135_2A19_20190522_0050_12s_to18s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 396900
stop in frames:  529200
135_2A19_20190522_0050_18s_to24s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 529200
stop in frames:  661500
135_2A19_20190522_0050_24s_to30s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 661500
stop in frames:  793800
135_2A19_20190522_0050_30s_to36s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 793800
stop in frames:  926100
135_2A19_20190522_0050_36s_to42s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 926100
stop in frames:  1058400
135_2A19_20190522_0050_42s_to48s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start i

start in frames 8467200
stop in frames:  8599500
135_2A19_20190522_0050_384s_to390s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 8599500
stop in frames:  8731800
135_2A19_20190522_0050_390s_to396s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 8731800
stop in frames:  8864100
135_2A19_20190522_0050_396s_to402s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 8864100
stop in frames:  8996400
135_2A19_20190522_0050_402s_to408s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 8996400
stop in frames:  9128700
135_2A19_20190522_0050_408s_to414s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 9128700
stop in frames:  9261000
135_2A19_20190522_0050_414s_to420s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 9261000
stop in frames:  9393300
135_2A19_20190522_0050_420s_to426s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 9393300
stop in frames:  9525600
135_2A19_20190522_0050_426s_to432s
(259, 64, 1)


start in frames 16802100
stop in frames:  16934400
135_2A19_20190522_0050_762s_to768s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 16934400
stop in frames:  17066700
135_2A19_20190522_0050_768s_to774s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 17066700
stop in frames:  17199000
135_2A19_20190522_0050_774s_to780s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 17199000
stop in frames:  17331300
135_2A19_20190522_0050_780s_to786s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 17331300
stop in frames:  17463600
135_2A19_20190522_0050_786s_to792s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 17463600
stop in frames:  17595900
135_2A19_20190522_0050_792s_to798s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 17595900
stop in frames:  17728200
135_2A19_20190522_0050_798s_to804s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 17728200
stop in frames:  17860500
135_2A19_20190522_0050_804s_to81

start in frames 25137000
stop in frames:  25269300
135_2A19_20190522_0050_1140s_to1146s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25269300
stop in frames:  25401600
135_2A19_20190522_0050_1146s_to1152s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25401600
stop in frames:  25533900
135_2A19_20190522_0050_1152s_to1158s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25533900
stop in frames:  25666200
135_2A19_20190522_0050_1158s_to1164s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25666200
stop in frames:  25798500
135_2A19_20190522_0050_1164s_to1170s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25798500
stop in frames:  25930800
135_2A19_20190522_0050_1170s_to1176s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25930800
stop in frames:  26063100
135_2A19_20190522_0050_1176s_to1182s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 26063100
stop in frames:  26195400
135_2A19_20190522_

...

135_2A19_20190522_0330_1134s_to1140s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25137000
stop in frames:  25269300
135_2A19_20190522_0330_1140s_to1146s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25269300
stop in frames:  25401600
135_2A19_20190522_0330_1146s_to1152s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25401600
stop in frames:  25533900
135_2A19_20190522_0330_1152s_to1158s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25533900
stop in frames:  25666200
135_2A19_20190522_0330_1158s_to1164s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25666200
stop in frames:  25798500
135_2A19_20190522_0330_1164s_to1170s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25798500
stop in frames:  25930800
135_2A19_20190522_0330_1170s_to1176s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 25930800
stop in frames:  26063100
135_2A19_20190522_0330_1176s_to1182s
(259, 64, 1)
(1, 259, 64, 1)
(1,

135_2A19_20190522_0330_1506s_to1512s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 33339600
stop in frames:  33471900
135_2A19_20190522_0330_1512s_to1518s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 33471900
stop in frames:  33604200
135_2A19_20190522_0330_1518s_to1524s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 33604200
stop in frames:  33736500
135_2A19_20190522_0330_1524s_to1530s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 33736500
stop in frames:  33868800
135_2A19_20190522_0330_1530s_to1536s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 33868800
stop in frames:  34001100
135_2A19_20190522_0330_1536s_to1542s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 34001100
stop in frames:  34133400
135_2A19_20190522_0330_1542s_to1548s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 34133400
stop in frames:  34265700
135_2A19_20190522_0330_1548s_to1554s
(259, 64, 1)
(1, 259, 64, 1)
(1,

start in frames 41409900
stop in frames:  41542200
135_2A19_20190522_0330_1878s_to1884s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 41542200
stop in frames:  41674500
135_2A19_20190522_0330_1884s_to1890s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 41674500
stop in frames:  41806800
135_2A19_20190522_0330_1890s_to1896s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 41806800
stop in frames:  41939100
135_2A19_20190522_0330_1896s_to1902s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 41939100
stop in frames:  42071400
135_2A19_20190522_0330_1902s_to1908s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 42071400
stop in frames:  42203700
135_2A19_20190522_0330_1908s_to1914s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 42203700
stop in frames:  42336000
135_2A19_20190522_0330_1914s_to1920s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 42336000
stop in frames:  42468300
135_2A19_20190522_

start in frames 49612500
stop in frames:  49744800
135_2A19_20190522_0330_2250s_to2256s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 49744800
stop in frames:  49877100
135_2A19_20190522_0330_2256s_to2262s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 49877100
stop in frames:  50009400
135_2A19_20190522_0330_2262s_to2268s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 50009400
stop in frames:  50141700
135_2A19_20190522_0330_2268s_to2274s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 50141700
stop in frames:  50274000
135_2A19_20190522_0330_2274s_to2280s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 50274000
stop in frames:  50406300
135_2A19_20190522_0330_2280s_to2286s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 50406300
stop in frames:  50538600
135_2A19_20190522_0330_2286s_to2292s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 50538600
stop in frames:  50670900
135_2A19_20190522_

start in frames 57815100
stop in frames:  57947400
135_2A19_20190522_0330_2622s_to2628s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 57947400
stop in frames:  58079700
135_2A19_20190522_0330_2628s_to2634s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 58079700
stop in frames:  58212000
135_2A19_20190522_0330_2634s_to2640s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 58212000
stop in frames:  58344300
135_2A19_20190522_0330_2640s_to2646s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 58344300
stop in frames:  58476600
135_2A19_20190522_0330_2646s_to2652s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 58476600
stop in frames:  58608900
135_2A19_20190522_0330_2652s_to2658s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 58608900
stop in frames:  58741200
135_2A19_20190522_0330_2658s_to2664s
(259, 64, 1)
(1, 259, 64, 1)
(1, 259, 45, 1)
start in frames 58741200
stop in frames:  58873500
135_2A19_20190522_

In [24]:
print(d_array[0][1].shape)

(1, 259, 3)


In [16]:
sum(d_array[:,1])

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 2.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 2.],
        [0., 0., 2.],
        [0., 0., 2.],
        [0., 0., 1.],
        [0., 0., 3.],
        [0., 0., 3.],
        [0., 0., 3.],
        [0., 0., 4.],
        [0., 0., 3.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 2.],
        [0., 0., 2.],
        [0., 0., 3.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 2.],
        [0., 0., 3.],
        [0., 0., 3.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 3.],
        [0., 0., 3.],
        [0

array(['135_2A19_20190522_0330_0s_to6s',
       array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
      